In [37]:
import yaml
import requests
import base64
import json
from pymongo import MongoClient
import time
from datetime import datetime,timedelta
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

In [38]:
sid = SentimentIntensityAnalyzer()

In [39]:
def load_api_key(filename='twitter_api_key.yaml'):
    """Load Yelp API client ID and client secret and return them as a dictionary."""
    with open(filename) as f:
        return yaml.load(f)
    
key = load_api_key()

In [40]:
client_key = key['consumer_key']
client_secret = key['consumer_secret']

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [41]:
base_url = 'https://api.twitter.com/'
auth_url = '{}oauth2/token'.format(base_url)

auth_headers = {
    'Authorization': 'Basic {}'.format(b64_encoded_key),
    'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
}

auth_data = {
    'grant_type': 'client_credentials'
}

auth_resp = requests.post(auth_url, headers=auth_headers, data=auth_data)

In [42]:
auth_resp.status_code

200

In [43]:
auth_resp.json().keys()

dict_keys(['token_type', 'access_token'])

In [44]:
access_token = auth_resp.json()['access_token']

In [45]:
search_headers = {
    'Authorization': 'Bearer {}'.format(access_token)    
}

In [86]:
url = 'https://api.twitter.com/1.1/application/rate_limit_status.json'

search_resp = requests.get(url, headers=search_headers)

In [87]:
json.loads(search_resp.content)['resources']['search']

{'/search/tweets': {'limit': 450, 'remaining': 403, 'reset': 1553814845}}

In [77]:
healthy = pd.read_csv('data/healthy.csv', header = None)

healthy_foods = list(healthy[0])
healthy_foods[35:]

In [85]:
unhealthy = pd.read_csv('data/unhealthy.csv', header = None)

unhealthy_foods = list(unhealthy[0])
unhealthy_foods[40:]

['cream cheese',
 'brownies',
 'cereal',
 'nutella',
 'quesadilla',
 'milkshake',
 'churro',
 'ramen',
 'fried chicken',
 'french toast']

In [64]:
req_count

369

In [30]:
client = MongoClient()
db = client['capstone']
coll=db['healthy']

In [52]:
#words = ['safeway','albertsons','ralphs','kroger','trader joe\'s','whole foods','jewel-osco', 'pavilions', 'food 4 less','randalls','tom thumb','star market','vons','united supermarkets','acme markets', 'carrs']
words = healthy_foods[16:]

req_count = 0
for query in words:
    print("starting: ",query)
    current = datetime.now()
    Prior = datetime.now() - timedelta(days=2)
    c = 0
    lst = []
    s = []
    q = query + " -filter:retweets"
    while current>Prior:
        if c == 0:
            search_params = {
                'q': q,
                'lang':'en',
                'count': 100,
                'tweet_mode':'extended'
                #,'geocode':"41.878485,-87.627893,100mi"
            }

            search_url = '{}1.1/search/tweets.json'.format(base_url)

            search_resp = requests.get(search_url, headers=search_headers, params=search_params)
            
            Query = json.loads( search_resp.content )
            
            #Query = t.search.tweets(q=q,lang='en', count=100,tweet_mode='extended',geocode="41.878485,-87.627893,100mi")['statuses']
            #Query = t.search.tweets(q=q,result_type='recent',count=100,tweet_mode='extended')
            next_cursor = Query['search_metadata']['next_results']
            c+=1
        else:
            search_params = {
                'q': q,
                'lang':'en',
                'count': 100,
                'tweet_mode':'extended',
                'max_id': max_id
                #,'geocode':"41.878485,-87.627893,100mi",
            }

            search_url = '{}1.1/search/tweets.json'.format(base_url)

            search_resp = requests.get(search_url, headers=search_headers, params=search_params)
            
            Query = json.loads( search_resp.content )
                        
            #Query = t.search.tweets(q=q,result_type='recent',count=100,max_id=max_id,tweet_mode='extended')
            next_cursor = Query['search_metadata']['next_results']
            c+=1
            
        for i,tweet in enumerate(Query['statuses']):
            if tweet['id'] not in s and query in tweet['full_text'].lower():
                date = tweet['created_at']
                current = datetime.strptime(date, '%a %b %d %H:%M:%S +0000 %Y')
                lst.append(tweet['full_text'])
                s.append(tweet['id'])
                ss = sid.polarity_scores(tweet['full_text'])
                coll.insert_one({'full_data': tweet,'text': tweet['full_text'],'keyword':query, 'sentiment':ss})
        req_count+=1
        if req_count % 50 == 0:
            print("current request count: ", req_count)
        max_id = tweet['id']
        if req_count>448:
            print("sleep time")
            time.sleep(60*15)
            req_count = 0
print("finished pull")

starting:  eggplant
starting:  fish
current request count:  150
current request count:  200
current request count:  250
current request count:  300
current request count:  350
current request count:  400
sleep time
current request count:  50
current request count:  100
current request count:  150
current request count:  200
current request count:  250
starting:  kale
current request count:  300
starting:  kebab
starting:  lamb
current request count:  350
current request count:  400
starting:  lentils
starting:  lettuce
sleep time
starting:  hummus
current request count:  50
starting:  mango
current request count:  100
current request count:  150
starting:  meat
current request count:  200
current request count:  250
current request count:  300
current request count:  350
current request count:  400
sleep time
current request count:  50
starting:  melon
current request count:  100
starting:  milk
current request count:  150
current request count:  200
current request count:  250
current

KeyboardInterrupt: 

In [53]:
coll.find({'keyword': 'milk'}).count()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


20887

In [32]:
max_id

1111303715758239745